# AutoEncoder - Dimensionality Reduction of MNIST FASHION DataSet
Compare the lossy ratio between multiple latent spaces

In [ ]:
%pip install -q imageio pydot tensorflow-gpu==2.9.1 keras matplotlib graphviz moviepy


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import os.path
import PIL
import PIL.Image
import keras_tuner as kt


## Import all Modules and Configure GPU

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist, fashion_mnist
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable float16 bit for improved performance.
        mixed_precision.set_global_policy('mixed_float16')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [ ]:
print("TensorFlow version:", tf.__version__)


## Load DataSet

In [ ]:
def loadDataFashion():
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

    TrainX = np.concatenate((train_images, test_images))
    # Normalize byte [0,255] -> [0,1]
    TrainX = (TrainX / 255.0)

    return TrainX

    #
def loadMNIST():
    (train_X, train_y), (test_X, test_y) = mnist.load_data()
    # Normalize byte [0,255] -> [0,1]
    train_X = train_X / 255.0
    test_X = test_X / 255.0

    return np.concatenate((train_X, test_X))


In [ ]:
EPOCHS = 192
BATCH_SIZE = 1024

fashionX = loadDataFashion()
MnistX = loadMNIST()

trainX = np.concatenate((fashionX, MnistX)).astype(dtype='float16')


## Presenting Example Data

In [ ]:
fig = plt.figure(figsize=(16, 6))
nrImage = 4 * 8
for index, image in enumerate(trainX[0:nrImage]):
    plt.subplot(4, 8, (index + 1))

    plt.imshow(X=((image + 1.0) / 2.0).astype(dtype='float32'), cmap=plt.cm.gray, aspect='auto')
    plt.axis("off")
plt.subplots_adjust(wspace=0.25, hspace=0.25)
plt.show(block=False)


# Create Dimensionality Reduction Model

In [ ]:
def model_constructor(hp, latent_space, image_size):

    class Autoencoder(keras.Model):
        def __init__(self, hp, **kwargs):
            super(Autoencoder, self).__init__(**kwargs)

            kernel_init = hp.Choice('kernel_initializer', ['uniform', 'lecun_uniform', 'normal', 'zero',
                                                           'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'])
            kernel_activation = hp.Choice('kernel_activation', ['relu', 'sigmoid', 'softmax', 'tanh'])

            number_layers = hp.Int(
                'number_layers', min_value=0, max_value=4, step=1)


            encoder_dense_layers = [layers.Dense(256 / (2 ** i), activation=kernel_activation, kernel_initializer=kernel_init)
                                    for i in range(0, number_layers + 1)]

            self.encoder = tf.keras.Sequential()
            self.encoder.add(layers.Flatten())

            for layer in encoder_dense_layers:
                self.encoder.add(layer)

            self.encoder.add(layers.Dense(latent_space, activation=kernel_activation, kernel_initializer=kernel_init, name="latent space"))

            decoder_dense_layers = [layers.Dense(256 / (2 ** i), activation=kernel_activation, kernel_initializer=kernel_init)
                                    for i in range(0, number_layers + 1)]

            self.decoder = tf.keras.Sequential()
            for layer in reversed(decoder_dense_layers):
                self.decoder.add(layer)

            self.decoder.add(layers.Dense(np.prod(image_size), activation='sigmoid', kernel_initializer=kernel_init))
            self.decoder.add(layers.Reshape(image_size))

        def call(self, x):
            encoded = self.encoder(x)
            decoded = self.decoder(encoded)
            return decoded

        def getEncoder(self):
            return self.encoder

        def getDecoder(self):
            return self.decoder

        def getModel(self):
            ae_input = layers.Input(shape=image_size, name="AE_input")
            ae_encoder_output = self.getEncoder()(ae_input)
            ae_decoder_output = self.getDecoder()(ae_encoder_output)

            return keras.Model(inputs=ae_input, outputs=ae_decoder_output)

        def compile(self, **kwargs):
            super(Autoencoder, self).compile(**kwargs)

        def summary(self, **kwargs):
            super(Autoencoder, self).summary(**kwargs)
            self.encoder.summary()

    autoencoder = Autoencoder(hp)
    hp_optimizer = hp.Choice('optimizer', ['sgd', 'adam', 'rmsprop'])

    autoencoder.compile(optimizer=hp_optimizer, loss='mse', metrics=['accuracy'])

    return autoencoder


## Hyperparameter
Find the best model out of all the parameters.

In [ ]:

image_shape = trainX[0].shape
print('X_train: ' + str(image_shape))

X_train, X_val, y_train, y_val = train_test_split(X_train, X_train, test_size=0.12, random_state=42)

latent_spaces = [1, 2, 4, 8, 16, 32, 64, 128]
ae_models = []

for latent_space in latent_spaces:

    def hyperparamter_model_builder(hp):
        model = model_constructor(hp, latent_space, image_shape)
        return model

    tuner = kt.Hyperband(hyperparamter_model_builder,
                         objective='val_accuracy',
                         max_epochs=16,
                         factor=3,
                         directory='cache',
                         project_name=str.format('MNIST AutoEncoder Reconstruct L{0} - {1}', latent_space, "Compression"))

    stop_early = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=4)
    tuner.search(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val),
                 callbacks=[stop_early], verbose=1)

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_models = tuner.get_best_models(4)

    autoencoder_model = tuner.hypermodel.build(best_hps)

    ae_models.append(autoencoder_model)


## Train Finalize Model

In [ ]:
def showResult(model, batchImage, maxNumImages=6):

    batch_iter = iter(batchImage)
    image_batch, _ = next(batch_iter)

    output = model.predict(batchImage)
    nrElements = min(len(output), maxNumImages)

    fig = plt.figure(figsize=(maxNumImages * 2, 2 * 2))
    for i in range(nrElements):
        trainImage = image

        ax = plt.subplot(2, maxNumImages, i + 1)
        plt.imshow((batchImage[i].astype(dtype='float32')), cmap='gray')
        plt.axis("off")

        ax = plt.subplot(2, maxNumImages, maxNumImages + i + 1)
        plt.imshow((output[i].astype(dtype='float32')), cmap='gray')
        plt.axis("off")

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    plt.close()
    return fig


class save_images(tf.keras.callbacks.Callback):

    def __init__(self, latent_space, trainData, **kwargs):
        super(tf.keras.callbacks.Callback, self).__init__(**kwargs)
        self.trainSet = trainData[0:20]

    def on_epoch_end(self, epoch, logs=None):
        fig = showResult(self.model, self.trainSet)
        fig.savefig("MNISTCompression{0}.png".format(epoch))
        fig = showResult(self.model, self.trainSet, 12)
        fig.savefig("MNISTCompressionBig{0}.png".format(epoch))


In [ ]:
model_results = []

for latent_space, ae_model in zip(latent_spaces, ae_models):

    # Construct the checkpoint path for specific latent space size.
    checkpoint_path = str.format("checkpoints/mnist_autoencoder_l{}_compression/cp.ckpt", latent_space)

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=0)

    checkpoint = tf.train.Checkpoint(model=autoencoder_model)
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path)).expect_partial()

    # The model weights (that are considered the best) are loaded into the model.
    if os.path.exists(checkpoint_path):
        autoencoder_model.load_weights(checkpoint_path)

    model_history = ae_model.fit(trainX, trainX,
                                 epochs=EPOCHS,
                                 batch_size=BATCH_SIZE,
                                 validation_split=0.1, shuffle=True,
                                 verbose=1, callbacks=[cp_callback, save_images(latent_space, trainX)])
    model_results.append(model_history)
    autoencoder_model.save_weights(checkpoint_path)


In [ ]:
# Plot Result
predictData = trainX[0:9]
for model in ae_models:
    result = model.predict(predictData)


In [ ]:

def plotCostHistories(results, loss_label="", val_label="", title="", x_label="", y_label=""):
    fig = plt.figure()
    for result in results:
        ax = plt.subplot(1, len(results))
        for k, v in result.history.items():
            plt.plot(v, label=k)
        plt.title(label=title)
        plt.ylabel(ylabel=y_label)
        plt.xlabel(xlabel=x_label)
        plt.legend(loc="upper left")

    plt.show(block=False)
    plt.savefig(title + ".png")


plotCostHistories(model_results, title=str.format(
    "CNN {0} Performance History", "Fashion"))

tf.keras.utils.plot_model(
    autoencoder_model, to_file=str.format('cnn_{0}_model.png', "Fashion"), show_shapes=True, show_dtype=True,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96,
    layer_range=None
)


## Model Neuron Networks

In [ ]:
for model in autoencoder_model:
    ae_model = model.getModel()
    ae_model.summary()

##